## Inside Schools Data Collection

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import dateutil.parser
import re
from time import sleep



Read List of school IDs from downloaded directory

In [3]:
school_nums = pd.read_csv('/Users/adamsantos/ds/metis/Regression School Project/2018_DOE_High_School_Directory.csv')
school_nums = school_nums['dbn']
school_codes = list(school_nums.values)
tester = school_nums[1]
#tester
#school_codes

Test connection

In [3]:
url = 'https://insideschools.org/school/14K454'

response = requests.get(url)

In [4]:
response.status_code

200

In [5]:
page = response.text
soup = BeautifulSoup(page, "lxml")

Function to clean strings

In [6]:
def clean_me(string):
    string = string.strip('\n')
    string = string.rstrip('\n')
    if string and string[-1] == '%':
        string = string.replace('%', '')
    try:
        string = float(string)
    except:
        string = string
    return string

Function to get data for a single school from soup object

In [4]:
def catch_em_all(soup, school_code):
    
    #Get base statistics
    titles = soup.find_all('div',{'class' : 'stat-title'})
    titles_final = [clean_me(x.text) for x in titles]
    
    values = soup.find_all('div',{'class' : 'stat-value'})
    values_final = [clean_me(x.text) for x in values]
    
    values_final = [x for x in values_final if not str(x).startswith('This school offers')]
    
    #Get teacher rating
    stars = soup.find_all('span',{'class' : 'stat-star icon-star gold'})
    rating = len(stars)
    titles_final.append('Teacher Effectiveness')
    values_final.append(rating)
    
    #Get school address
    address = soup.find('div',{'class' : 'location-address'}).text
    address = address.strip('\n')
    address = address.replace('\n',' ')
    address = ' '.join(address.split())
    titles_final.append('Address')
    values_final.append(address)
    
    #Put school code into final dict
    titles_final.append('School Code')
    values_final.append(school_code)
    
    name = soup.find('h1',{'class' : 'school-name'}).text
    name=name.strip('\n')
    name=name.rstrip('\n')
    titles_final.append('School Name')
    values_final.append(name)
    
    dictionary = dict(zip(titles_final, values_final))
    return dictionary


#test = catch_em_all(soup,'02M560')
#test

Get data from each school 

In [8]:
data=[]

for site in school_codes:
    url = 'https://insideschools.org/school/'+site
    response = requests.get(url)
    
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, "lxml")
    
        data.append(catch_em_all(soup,site))
    
    sleep(0.5)
  

In [9]:
df=pd.DataFrame(data)

df=df.dropna(thresh=100, axis=1)

df.head()

,Address,Are teachers effective?,Asian,Average daily attendance,Black,English language learners,Enrollment,Free or reduced priced lunch,Hispanic,How many English language learners graduate in 4 years?,...,How many teachers say they would recommend this school to other families?,Metal detectors?,Other,School Code,School Name,Shared campus?,Students with disabilities,Teacher Effectiveness,White,Years of principal experience at this school
0,1440 Story Avenue Bronx NY 10473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,08X519,Felisa Rincon de Gautier Institute for Law and...,,NaN,0,NaN,NaN
1,49 Flatbush Avenue Extension Brooklyn NY 11201,,1.0,81.0,77.0,3.0,221.0,86.0,19.0,NaN,...,73.0,No,1.0,13K350,Urban Assembly High School of Music and Art,,23.0,2,2.0,12.0
2,160-20 Goethals Avenue Jamaica NY 11432,,51.0,98.0,27.0,1.0,690.0,63.0,10.0,NaN,...,68.0,No,8.0,28Q680,Queens Gateway to Health Sciences Secondary Sc...,No,12.0,4,4.0,5.2
3,456 White Plains Road Bronx NY 10473,,11.0,75.0,41.0,21.0,281.0,93.0,33.0,56.0,...,88.0,No,4.0,08X282,Women's Academy of Excellence,,19.0,3,12.0,13.0
4,34-65 192nd Street Flushing NY 11358,,15.0,95.0,4.0,2.0,567.0,56.0,36.0,NaN,...,95.0,No,2.0,25Q285,World Journalism Preparatory: A College Board ...,,19.0,4,44.0,0.6


Save for future use

In [ ]:
df.to_pickle("oct_3.pkl")